In [ ]:
import subprocess
import glob
from tqdm import tqdm
from tile_operator.operate import TileOperate, file_to_bounds

In [ ]:
file_path = "./wajima.geojson"
bbox = file_to_bounds(file_path).bounds()
print(bbox)

to = TileOperate(
    bbox=bbox,
    zoom_level=18,
)

In [ ]:
from PIL import Image

# 画像サイズを半分にする関数
def half_size_img(img_file):
    # リサイズ前の画像を読み込み
    img = Image.open(img_file)
    # 読み込んだ画像の幅、高さを取得し半分に
    (width, height) = (img.width // 2, img.height // 2)
    # 画像をリサイズする
    img_resized = img.resize((width, height))
    # ファイルを保存
    img_file_resize = f".{img_file.split('.')[1]}.png"
    img_resized.save(img_file_resize)

    return img_file_resize

# 学習済みモデルを使って、建物の場所を着色

国土地理院（2023）：CNNによる地物抽出のための深層学習済モデル，国土地理院技術資料 H1-No.36．
https://gisstar.gsi.go.jp/gsi-dataset/99/index.html

全国最新写真（シームレス）
URL：https://cyberjapandata.gsi.go.jp/xyz/seamlessphoto/{z}/{x}/{y}.jpg

令和6年能登半島地震 輪島東地区　正射画像（2024年1月2日撮影）
URL：https://cyberjapandata.gsi.go.jp/xyz/20240102noto_wazimahigashi_0102do/{z}/{x}/{y}.png

In [ ]:
tile_list = [("seamlessphoto", "jpg")
             , ("20240102noto_wazimanaka_0102do", "png")]

In [ ]:
for tile_name, tile_format in tile_list:
    print(tile_name)
    files = []
    for y_name in tqdm(glob.glob(f"./{tile_name}/18/*")):
        # 学習済みモデルを用いた推論部分
        output_dir = f"./{tile_name}_bld/18/{y_name.split('/')[-1]}/"
        command = ["python3", "./inference512.py"
                   , "--input_dir", f"{y_name}/"
                   , "--output_dir", output_dir
                   , "--model", "./model-09/" # 今回は普通建物のモデル
                   , "--ngf", "64"
                   , "--output_filetype", "png"]
        subprocess.run(command)

        for y_name_2 in tqdm(glob.glob(f"{output_dir}*.png")):
            y_name_3 = half_size_img(y_name_2) # 地図タイルが512×512になっているので256×256へ変更
            to.tile_to_geotiff(y_name_3) # 地図タイルをgeotiffへ変換
        
        files = files + glob.glob(f"{output_dir}/*.tif")
    
    command = ["gdal_merge.py", "-o", f"./{tile_name}_bld.tif", "-co", "BIGTIFF=YES"] + files
    print(command)
    subprocess.run(command)

In [ ]:
# 画像を横結合
import cv2
im1 = cv2.imread(f'{tile_list[0][0]}_bld.tif') # 災害前
im2 = cv2.imread(f'{tile_list[1][0]}_bld.tif') # 災害後

im_h = cv2.hconcat([im1, im2])
cv2.imwrite('./before_after.jpg', im_h)